 # importing libraries

In [3]:
import numpy as np
import tensorflow as tf
import re #clean the text, to simplyfy conversation
import time #to measure training time

### part 1 - data preprocessing

### importing the dataset

In [4]:
# alternate method to open file

#import os
#path = '/home/raxit/chatbot/datasets'
#file = 'movie_lines.txt'
#lines = open(os.path.join(path,file))

lines = open('/home/raxit/chatbot/datasets/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('/home/raxit/chatbot/datasets/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')


In [5]:
#lines

In [6]:
#conversations

## creating a dictionary that maps each line and its id

In [7]:
id2line = {}
for line in lines :
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [8]:
#id2line

## creating the list of all conversations

In [9]:
conversations_ids = []
for conversation in conversations [:-1]:
    _conversation = conversation.split (' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(_conversation.split(','))

In [10]:
#conversations_ids



## getting separately questions and answers

In [11]:
questions = [ ]
answers = [ ]
for conversation in conversations_ids : 
    for i in range (len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [12]:
## questions


## first cleaning of text

In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"how's","how is",text)
    text = re.sub(r"\'re","are",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"\n't","not",text)
    text = re.sub(r"can't","cannot",text)
    text = re.sub(r"\'ll","will",text)
    text = re.sub(r"\'ve","have",text)
    text = re.sub(r"\'d","would",text)
    text = re.sub(r"didn't","did not",text)
    text = re.sub(r"[-()\"#*$%&!~|,.?{}+:;<>@/^]","",text)
    return text

## cleaning the questions

In [14]:
clean_questions = []
for question in questions:
    clean_questions.append (clean_text(question))

## cleaning the answers

In [15]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [16]:
#clean_questions

In [17]:
#clean_answers

## Filtering out the questions and answers that are too short or too long

In [18]:
short_questions = []
short_answers = []
i = 0
for question in clean_questions:
    if 2 <= len(question.split()) <= 25 :
        short_questions.append(question)
        short_answers.append(clean_answers[i])
    i += 1
    
clean_questions = []
clean_answers = []
i = 0
for answer in short_answers:
    if 2 <= len(answer.split()) <= 25:
        clean_answers.append(answer)
        clean_questions.append(short_questions[i])
    i += 1

## creating a dictionary that maps each word to its number of occurance in questions and in answers

In [17]:
word2count = {}  #making word2count dictionary

#for questions

for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count [word] = 1
        else:
            word2count [word] += 1
            
            
#now for answers-
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
        
        

## creating two dictionaries that maps the questions word and answers word to a unique integer
 

In [18]:
    
    #here we are taking a threshold of word occurance in questions and  in answers
  #  if any word is coming below threshold quantity we will exclude that and whose word
   # qunatity is greater than the threshold we will take that word.
    
threshold_questions = 15

In [19]:
#making dictionary for questions

questionswords2int = {}
word_number = 0 #again we are starting from zero
for word,count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
#making dictionary for answers
threshold_answers = 15
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1
        
            


## adding the last tokens to the directories

In [20]:
tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]

for token in tokens:
    questionswords2int[token] = len(questionswords2int) +1
    
for token in tokens:
    answerswords2int[token] = len(answerswords2int) +1

## creating the inverse dictionary for answersword2int dictionary to map the words

In [21]:
# w - word & w_i - word integer
answersints2word = {w_i:w for w,w_i in answerswords2int.items()}

In [22]:
#answersints2word

## adding the end of string token to the end of evry answers

In [23]:
# doing beacause EOS needed in the end of decoding in seq2seq model


for i in range (len(clean_answers)):
    clean_answers[i] += ' <EOS>'

In [24]:
#clean_answers

## translating all the question and answers in to integer

In [25]:
#and replacing all the words that were filtered out by <OUT>
# to sort all the questions and answers by their length to optimize training performance

In [26]:
#list for changing the words of each question into integer value

questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)

## list for changing the words of each answer into integer value

In [27]:
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    answers_into_int.append(ints)

In [28]:
#answers_into_int

In [29]:
#questions_into_int

## sorting question and answers by the length of question

In [30]:
#it will reduce the padding during the training and speedup our training


sorted_clean_questions = []
sorted_clean_answers = []
for length in range (1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [31]:
#sorted_clean_questions

In [32]:
#sorted_clean_answers


# BUILDING SEQ2SEQ MODEL...

## Creating placeholders for the inputs and targets

In [33]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = "input")#create placeholderr for inputs
    targets = tf.placeholder(tf.int32,[None,None], name  = "target")#create placeholder for targets
# making parameter(hyperparameter[learning_rate & keep_prob]) used to control dropout rate
# dropouts are the rate of neurons that we are choosed to overwrite during one iteration in training
# we are using 20% dropout to deactivate 20% neuron
# we increase or decrease dropout by using keep_prob hyperparameter
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

## preprocessing the targets,

In [34]:
#here we'll make the batches of the targets(answers[must be in batches])
# because decoder will decode the batches.(neural nets wont accept the single target)

#sentence must start from the SOS(start of the sentence), so we have to make SOS in each answers of each batches to proceed.

#so here we create batches and adding SOS tokens.

In [35]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0],[batch_size, -1],[1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets
    #batch_size is no. of lines in batch
    #strided_slice - extract the subset of tensors
    # for horizontal, axis=1 & for vertical, axis=0 

# creating the encoder rnn layer

In [36]:
def encoder_rnn (rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]*num_layers)
    # num_layers create encoder cell
    # encoder cell consist of several lstm layers
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    # bidirectional_dynamic_rnn take input and build independent forward and backward rnn(input of forward and backward must match)
    # _, -indicates that we only want second element returned by this future bidirectional dinemic rnn function
    return encoder_state

## decoding the training set

In [37]:
#for encoder we only need encoder state and for decoder we only need decoder output



In [38]:
def decode_training_set(encoder_state,
                        decoder_cell,
                        decoder_embedded_input,
                        sequence_length,
                        decoding_scope,
                        output_function,
                        keep_prob,
                        batch_size):
#decoder getting encoder_state as a part of input proceed to decoding
#decoder_cell - cell in the rnn of the decoder
#embeddings are useful to reduce the dimensoility of categorical variables
#embeddings - is a mapping from descrete objects(i.e. words) to vectors of real no.s(uniquiley) 
#here we roviding i/p to embeddings
#decoding_scope - advance data structure that wraps TF variables  

    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    #we are processing 3 dimn matrix
    
    #preprocesses the training data
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    # attention_keys  = key to compare with target states
    #attention_values = values to construct the context vector(return by encoder and used by decoder)
    #attention_score_function - used to compute similarity between keys and target_states
    # attentio construct_function - used to built attention state
    
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")#this is the name scope for the decoder function
    
    
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder (decoder_cell,
                                                                                                               training_decoder_function,
                                                                                                               decoder_embedded_input,
                                                                                                               sequence_length,
                                                                                                               scope = decoding_scope)
    #now we take the droput of the decoder_output
    
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

## decoding the test/validation set

In [39]:
# we will not use dropout for the test set predictions
# here we are using inference function to predict the outcome that is not involved in  
#training although we dont backpropagate this,cause we are decoding test set to predict

In [40]:
def decode_test_set(encoder_state,
                    decoder_cell,
                    decoder_embeddings_matrix,
                    sos_id,
                    eos_id,
                    maximum_length,
                    num_words,
                    decoding_scope,
                    output_function,
                    keep_prob,
                    batch_size):
    
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    
    
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = 'bahdanau', num_units = decoder_cell.output_size)
    
    
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")#this is the name scope for the decoder function
    
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
                                                                                              
                                                                                              
                                                                                              
                                                                                            
                                                                  
    return test_predictions
    

## creating the decoder rnn

In [41]:
def decoder_rnn(decoder_embedded_input,
                decoder_embeddings_matrix,
                encoder_state,
                num_words,
                sequence_length,
                rnn_size,
                num_layers,
                word2int,
                keep_prob,
                batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        #create lstm layer
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        #now apply some dropout regulariztion to reduce overfitting and increase the accuracy
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1) #stddev = standard deviation
        biases = tf.zeros_initializer()
        output_function = lambda X: tf.contrib.layers.fully_connected(X,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix, 
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
        
        return training_predictions, test_predictions

# building the final seq2seq model

In [42]:
def seq2seq_model(inputs,
                  targets,
                  keep_prob,
                  batch_size,
                  sequence_length,
                  answers_num_words,
                  questions_num_words,
                  encoder_embedding_size,
                  decoder_embedding_size,
                  rnn_size,
                  num_layers,
                  questionswords2int):
    
    
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix, 
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    
    
    return training_predictions, test_predictions

# training the seq2seq model

## setting the Hyperparameter

In [43]:
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0002
keep_probability = 0.5

## defining the session

In [44]:
tf.reset_default_graph()
session = tf.InteractiveSession()

## loading the model inputs

In [45]:
inputs, targets, lr, keep_prob = model_inputs()

## setting a sequence_length

In [46]:
#here we are gonna set a default value to produce when the output is not fed into the RNN

sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

## getting the shape of the Inputs tensor

In [47]:
input_shape = tf.shape(inputs)

## getting the training and test predictions

In [48]:
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)

## setting up the loss error, the optimizer and gradient clipping

In [50]:
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    #ones function will take only one argument followed by no.of lines and then columns
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    #now clipp all gradients and compute them
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.),grad_variable)for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

## padding the sequence with the <PAD  token

In [53]:
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']]*(max_sequence_length-len(sequence)) for sequence in batch_of_sequences]

## splitting the data into batches of questions and answers

In [54]:
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch   #yield - to return the value of function
        

## splitting the questions and answers into training and validation sets

In [55]:
training_validation_split = int(len(sorted_clean_questions) * 0.15)

training_questions = sorted_clean_questions[training_validation_split : ]
training_answers = sorted_clean_answers[training_validation_split : ]

validation_questions = sorted_clean_questions[ : training_validation_split]
validation_answers = sorted_clean_answers[ : training_validation_split]


# Training

In [ ]:
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 1000
checkpoint = "chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range (1, epochs+1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch, 
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds' .format(epoch,
                                                                                                                                        epochs,
                                                                                                                                        batch_index,
                                                                                                                                        len(training_questions) // batch_size,
                                                                                                                                        total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                        int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_training_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print('Sorry I do not speak better, I need to practice more.')
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print('My apologies, I cannot speak better anymore. This is the best I can do!.')
        break
print('Game Over')
                

# Testing the seq2seq model

## loading the weights and running the session

In [ ]:
checkpoint = "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.saver()
saver.restore(session, checkpoint)

## converting the questions from strings to lists of encoding integers

In [ ]:
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int[''])for word in question.split()]

## setting up the chat

In [ ]:
while(True):
    question = input ("you: ")
    if question == "Goodbye":
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (25 - len(question))
    fake_batch = np.zeros((batch_size, 25))
    fake_batch[0] = question
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if answersints2word[i] == 'i':
            token = ' I'
        elif answersints2word[i] == '<EOS>':
            token = '.'
        elif answersints2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersints2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)